In [ ]:
import os
from keras.models import Model
from keras.layers import Input,Dense,ReLU,Dropout,Softmax,Conv2D,MaxPool2D,Lambda,GaussianNoise
from keras.layers import Bidirectional,Flatten,CuDNNGRU
from keras.utils.vis_utils import plot_model

def ICAMC(weights=None,
             input_shape=[2,1024],
             classes=26,
             **kwargs):
    if weights is not None and not (os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), '
                         'or the path to the weights file to be loaded.')
    dr = 0.45
    input = Input(input_shape+[1],name='input')
    x=Conv2D(32,(1,8), activation="relu", name="conv1", padding='same', kernel_initializer='glorot_uniform')(input)
    x= MaxPool2D(pool_size=(2, 2))(x)
    x=Conv2D(32,(1,4), activation="relu", name="conv2", padding='same', kernel_initializer='glorot_uniform')(x)
    x=Conv2D(64,(1,8),activation="relu", name="conv3", padding='same', kernel_initializer='glorot_uniform')(x)
    x = Dropout(dr)(x)
    x=Conv2D(64,(1,8), activation="relu", name="conv4", padding='same', kernel_initializer='glorot_uniform')(x)
    x = Dropout(dr)(x)
    x=Flatten()(x)
    x = Dense(64,activation='relu',name='dense1')(x)
    x=Dropout(dr)(x)
    x = GaussianNoise(1)(x)
    x = Dense(26,activation='softmax',name='dense2')(x)

    model = Model(inputs = input,outputs = x)

    # Load weights.
    if weights is not None:
        model.load_weights(weights)

    return model

import keras
if __name__ == '__main__':
    model =  ICAMC(None,input_shape=[2,1024],classes=26)

    adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

    print('models layers:', model.layers)
    print('models config:', model.get_config())
    print('models summary:', model.summary())

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import itertools
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, mean_absolute_error, r2_score

# Show loss curves
def show_history(history):
    plt.figure()
    plt.title('Training loss performance')
    plt.xlabel('Epoch', fontdict={'size':16,})
    plt.ylabel('Loss', fontdict={'size':16,})
    plt.plot(history.epoch, history.history['loss'], label='train loss', marker='>', linewidth=2)
    plt.plot(history.epoch, history.history['val_loss'], label='validation loss', marker='*', linewidth=2)
    plt.legend()
    plt.savefig('figure/total_loss.pdf', format='pdf', dpi=1200, bbox_inches = 'tight')
    plt.close()

def plot_confusion_matrix(cm, title='', cmap=plt.get_cmap("Blues"), labels=[], save_filename=None):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm*100, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=10)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=90, size=12)
    plt.yticks(tick_marks, labels, size=12)
    for i in range(len(tick_marks)):
        for j in range(len(tick_marks)):
            if i != j:
                text = plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10)
            elif i == j:
                color = 'darkorange' if int(np.around(cm[i,j]*100)) == 100 else 'darkorange'
                text = plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10, color=color)

    plt.tight_layout()
    plt.ylabel('True label', fontdict={'size':16,})
    plt.xlabel('Predicted label', fontdict={'size':16,})
    if save_filename is not None:
        plt.savefig(save_filename, format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

def calculate_confusion_matrix(Y, Y_hat, classes):
    n_classes = len(classes)
    conf = np.zeros([n_classes, n_classes])
    confnorm = np.zeros([n_classes, n_classes])

    for k in range(0, Y.shape[0]):
        i = list(Y[k,:]).index(1)
        j = int(np.argmax(Y_hat[k,:]))
        conf[i,j] = conf[i,j] + 1

    for i in range(0, n_classes):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    right = np.sum(np.diag(conf))
    wrong = np.sum(conf) - right
    return confnorm, right, wrong

def calculate_acc_at1snr_from_cm(cm):
    return np.round(np.diag(cm) / np.sum(cm, axis=1), 3)

def calculate_metrics(Y, Y_hat):
    Y_true = np.argmax(Y, axis=1)
    Y_pred = np.argmax(Y_hat, axis=1)
    accuracy = accuracy_score(Y_true, Y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(Y_true, Y_pred, average='weighted')
    mse = mean_squared_error(Y_true, Y_pred)
    mae = mean_absolute_error(Y_true, Y_pred)
    r2 = r2_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

def calculate_acc_cm_each_snr(Y, Y_hat, Z, classes=None, save_figure=True, min_snr=0):
    Z_array = Z[:, 0]
    snrs = sorted(list(set(Z_array)))
    acc = np.zeros(len(snrs))
    acc_mod_snr = np.zeros((len(classes), len(snrs)))

    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }

    i = 0
    for snr in snrs:
        Y_snr = Y[np.where(Z_array == snr)]
        Y_hat_snr = Y_hat[np.where(Z_array == snr)]

        cm, right, wrong = calculate_confusion_matrix(Y_snr, Y_hat_snr, classes)
        accuracy, precision, recall, f1 = calculate_metrics(Y_snr, Y_hat_snr)

        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)

        if snr >= min_snr:
            plot_confusion_matrix(cm, cmap=plt.cm.Blues, labels=classes, save_filename='figure/cm_snr{}.pdf'.format(snr))

        acc[i] = round(1.0 * right / (right + wrong), 3)
        print('Accuracy at %ddb: %.2f%s / (%d + %d)' % (snr, 100*acc[i], '%', right, wrong))
        acc_mod_snr[:, i] = calculate_acc_at1snr_from_cm(cm)
        i += 1

    fd = open('acc_overall_128k_on_512k_wts.dat', 'wb')
    pickle.dump(('128k', '512k', acc), fd)
    fd.close()

    # Plot Accuracy, Precision, Recall, F1-Score
    for metric, values in metrics.items():
        plt.figure(figsize=(8, 6))
        plt.plot(snrs, values, label=metric)
        for x, y in zip(snrs, values):
            plt.text(x, y, round(y, 3), ha='center', va='bottom', fontsize=8)
        plt.xlabel("Signal to Noise Ratio")
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} vs SNR")
        plt.legend()
        plt.grid()
        plt.savefig(f'figure/{metric}_vs_snr.pdf', format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()

    # Plot recognition accuracy for each modulation at different SNRs
    dis_num = 7
    marker = itertools.cycle(('D', '*', 'o' , '>', '<', 'P', 'H'))
    for g in range(int(np.ceil(acc_mod_snr.shape[0] / dis_num))):
        beg_index = g * dis_num
        end_index = np.min([(g + 1) * dis_num, acc_mod_snr.shape[0]])
        plt.figure(figsize=(12, 10))
        plt.xlabel("Signal to Noise Ratio (dB)", fontdict={'size': 20,})
        plt.ylabel("Recognition Accuracy (%)", fontdict={'size': 20,})
        for i in range(beg_index, end_index):
            plt.plot(snrs, acc_mod_snr[i], label=classes[i], marker=next(marker), linewidth=3, markersize=12)
        plt.legend(fontsize=24)
        plt.grid()
        if save_figure:
            plt.savefig('figure/acc_with_mod_{}.pdf'.format(g + 1), format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()

In [ ]:
import random
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import sys,h5py
import tensorflow as tf

import pandas as pd
from keras.utils.np_utils import to_categorical
classes = ['BPSK',
               'QPSK',
               '8PSK',
               '16PSK',
               '32PSK',
               '64PSK',
               '4QAM',
               '8QAM',
               '16QAM',
               '32QAM',
               '64QAM',
               '128QAM',
               '256QAM',
               '2FSK',
               '4FSK',
               '8FSK',
               '16FSK',
               '4PAM',
               '8PAM',
               '16PAM',
               'AM-DSB',
               'AM-DSB-SC',
               'AM-USB',
               'AM-LSB',
                'FM',
                'PM']

In [ ]:
##traindata
data1 = h5py.File('Dataset/HisarMod2019.1/Train/train.mat','r')
train=data1['data_save'][:]
train=train.swapaxes(0,2)

data2 = h5py.File('Dataset/HisarMod2019.1/Test/test.mat','r')
test=data2['data_save'][:]
test=test.swapaxes(0,2)
train=np.expand_dims(train,axis=3)
test=np.expand_dims(test,axis=3)

In [ ]:
##label
train_labels = pd.read_csv('Dataset/HisarMod2019.1/Train/train_labels1.csv',header=None)
train_labels=np.array(train_labels)
train_labels = to_categorical(train_labels, num_classes=None)

test_labels = pd.read_csv('Dataset/HisarMod2019.1/Test/test_labels1.csv',header=None)
test_labels =np.array(test_labels)
test_labels = to_categorical(test_labels, num_classes=None)

In [ ]:
##snr
train_snr=pd.read_csv('Dataset/HisarMod2019.1/Train/train_snr.csv',header=None)
train_snr=np.array(train_snr)

test_snr=pd.read_csv('Dataset/HisarMod2019.1/Test/test_snr.csv',header=None)
test_snr=np.array(test_snr)

In [ ]:
n_examples = train.shape[0]
n_train = int(n_examples * 0.6)
n_val = int(n_examples * 0.15)
train_idx = list(np.random.choice(range(0, n_examples), size=n_train, replace=False))
val_idx = list(set(range(0, n_examples)) - set(train_idx))
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)
X_train = train[train_idx]
Y_train = train_labels[train_idx]
X_val = train[val_idx]
Y_val = train_labels[val_idx]
X_test = test
Y_test = test_labels
Z_test = test_snr

In [ ]:
# Set up some params
nb_epoch = 200     # number of epochs to train on
batch_size = 300  # training batch size

In [ ]:
model=ICAMC()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
plot_model(model, to_file='model.png',show_shapes=True) # print model
model.summary()

In [ ]:
filepath = 'weights/weights.h5'
import time
TRS_PROPOSED=time.time()

In [ ]:
history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
    validation_data=(X_val,Y_val),
    callbacks = [
                keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
                keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,verbose=1,patince=5,min_lr=0.0000001),
                keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

                ]
                    )

In [ ]:
TRE_PROPOSED=time.time()
T_PROPOSED=TRE_PROPOSED-TRS_PROPOSED
T_PROPOSED

In [ ]:
#Show simple version of performance
TES_PROPOSED=time.time()
score = model.evaluate(X_test, Y_test, verbose=1, batch_size=batch_size)
print(score)

In [ ]:
show_history(history)

In [ ]:
def predict(model):
    model.load_weights(filepath)
    # Plot confusion matrix
    test_Y_hat = model.predict(X_test, batch_size=batch_size)
    cm, right, wrong = calculate_confusion_matrix(Y_test, test_Y_hat, classes)
    acc = round(1.0 * right / (right + wrong), 4)
    print('Overall Accuracy:%.2f%s / (%d + %d)' % (100 * acc, '%', right, wrong))
    plot_confusion_matrix(cm, labels=['BPSK',
                                      'QPSK',
                                      '8PSK',
                                      '16PSK',
                                      '32PSK',
                                      '64PSK',
                                      '4QAM',
                                      '8QAM',
                                      '16QAM',
                                      '32QAM',
                                      '64QAM',
                                      '128QAM',
                                      '256QAM',
                                      '2FSK',
                                      '4FSK',
                                      '8FSK',
                                      '16FSK',
                                      '4PAM',
                                      '8PAM',
                                      '16PAM',
                                      'AM-DSB',
                                      'AM-DSB-SC',
                                      'AM-USB',
                                      'AM-LSB',
                                      'FM',
                                      'PM'], save_filename='figure/total_confusion.png')
    calculate_acc_cm_each_snr(Y_test, test_Y_hat, Z_test, classes, min_snr=-18)

In [ ]:
predict(model)

In [ ]:
TEE_PROPOSED=time.time()
TE_PROPOSED=TEE_PROPOSED-TES_PROPOSED